In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import dices
import yaml
import healpy as hp
import heracles
import camb
import camb.correlations
import skysegmentor
from camb.correlations import gauss_legendre_correlation as glc

In [2]:
with open("./dices_conf.yaml", mode="r") as file:
    config = yaml.safe_load(file)

In [3]:
config

{'Nside': 256,
 'Njk': 50,
 'Output': {'save': False, 'path': './tests/'},
 'Fields': {'Nbins': 2,
  'TomoBin': 'TOM_BIN_ID',
  'Pos': {'Ra': 'SHE_RA', 'Dec': 'SHE_DEC'},
  'She': {'Ra': 'SHE_RA',
   'Dec': 'SHE_DEC',
   'E1': 'SHE_E1_CAL',
   'E2': '-SHE_E2_CAL',
   'Weights': 'SHE_WEIGHT'}},
 'bins': {'Lmin': 10, 'Lmax': 256, 'Lbin': 10, 'Logbins': True},
 'DICES': {'bin': False,
  'mask_correction': True,
  'shrinkage': True,
  'delete2_correction': True}}

In [4]:
POS1 = heracles.read_maps("gaussian_sims/gaussian_sim_1/POS_1.fits")
SHE1 = heracles.read_maps("gaussian_sims/gaussian_sim_1/SHE_1.fits")
POS2 = heracles.read_maps("gaussian_sims/gaussian_sim_1/POS_2.fits")
SHE2 = heracles.read_maps("gaussian_sims/gaussian_sim_1/SHE_2.fits")

In [5]:
data_maps = {}
data_maps[("POS", 1)] = POS1[("POS", 1)]
data_maps[("POS", 2)] = POS2[("POS", 2)]
data_maps[("SHE", 1)] = SHE1[("SHE", 1)]
data_maps[("SHE", 2)] = SHE2[("SHE", 2)]

vis_maps = {}
vis_maps[("VIS", 1)] = POS1[('POS', 1)] / POS1[('POS', 1)]
vis_maps[("VIS", 2)] = POS1[('POS', 1)] / POS1[('POS', 1)]
vis_maps[("WHT", 1)] = POS1[('POS', 1)] / POS1[('POS', 1)]
vis_maps[("WHT", 2)] = POS1[('POS', 1)] / POS1[('POS', 1)]

jk_maps = {}
for key in list(vis_maps.keys()):
    vmap = vis_maps[key]
    jk_maps[key] = skysegmentor.segmentmapN(vmap, config["Njk"])

In [6]:
SHE1[("SHE", 1)]

array([[-0.00985523,  0.0091472 , -0.01290115, ..., -0.00209875,
         0.00083939, -0.00313652],
       [ 0.00739814, -0.00454436,  0.00516084, ...,  0.00161228,
        -0.00757816,  0.00723842]])

In [7]:
DICESObj = dices.DICES(data_maps,
                       jk_maps,
                       vis_maps, 
                       config) 

In [8]:
cls0, _ = DICESObj.get_cls()
for key in cls0.keys():
    cl = cls0[key].__array__()
    cls0[key] = heracles.Result(cl, DICESObj.ls)

In [9]:
cls1, _ = DICESObj.get_delete1_cls(mask_correction=False)

 - Biases for entire region: [0. 0.] [0. 0.]
Computing Ngal and Wmean/var for Jackknife regions
 - Biases for jackknife regions: [array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.])] [array([0., 0.]), ar

ValueError: too many values to unpack (expected 2)

In [ ]:
DICESObj.bin = False
shrunk_cov1, cov1, target = DICESObj._get_delete1_cov(cls0, cls1)

In [ ]:
cls2, _ = DICESObj.get_delete2_cls(mask_correction=False)

In [ ]:
Q = DICESObj._get_delete2_correction(
    cls0,
    cls1,
    cls2,
)
cov2 = {}
for key in list(cov1.keys()):
    cov2[key] = cov1[key] - Q[key]

In [ ]:
dices_cov = DICESObj._get_dices_cov(cls0, shrunk_cov1, cov2)

In [ ]:
cls0 = dices.compsep_Cls(cls0)
for key in list(cls1.keys()):
    cls1[key] = dices.compsep_Cls(cls1[key])
for key in list(cls2.keys()):
    cls2[key] = dices.compsep_Cls(cls2[key])

In [ ]:
_cls1 = {}
for key in list(cls1[1].keys()):
    _cls = []
    for i in range(1, DICESObj.JackNjk+1):
        cl = cls1[i][key]
        _cls.append(cl)
    _cls1[key] = np.array(_cls)

_cls2 = {}
for key in list(cls2[(1, 2)].keys()):
    _cls = []
    for i in range(1, DICESObj.JackNjk+1):
        for j in range(i+1, DICESObj.JackNjk+1):
            cl = cls2[(i, j)][key]
            _cls.append(cl)
    _cls2[key] = np.array(_cls)

cls1_m = {}
cls1_s = {}
cls2_m = {}
cls2_s = {}
for key in list(_cls1.keys()):
    cls1_m[key] = np.mean(_cls1[key], axis=0)
    cls1_s[key] = np.std(_cls1[key], axis=0)
    cls2_m[key] = np.mean(_cls2[key], axis=0)
    cls2_s[key] = np.std(_cls2[key], axis=0)

In [ ]:
ls = DICESObj.ls
lgrid = DICESObj.lgrid

In [ ]:
# WE HAVE TO FIX THIS
vcls = np.load('./data/validation_cls.npy', allow_pickle=True).item()
vcov = np.load('./data/validation_cov.npy', allow_pickle=True).item()
bls = np.load('./data/validation_bls.npy', allow_pickle=True).item()

In [ ]:
# Determine the number of subplots needed
num_plots = len(cls1_m)
num_cols = math.ceil(math.sqrt(num_plots))
num_rows = math.ceil(num_plots / num_cols)

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(25, 15))

# Flatten axes array for easy iteration
axes = axes.flatten()

# Plot each Cls with error bars
for (ax, key) in zip(axes, cls1_m.keys()):
    a, b, na, nb = key
    _vcl = vcls[key]
    _cl = cls0[key]#-bls[key]
    cl1m = cls1_m[key]
    cl1s = cls1_s[key]
    cl2m = cls2_m[key]
    cl2s = cls2_s[key]
    ax.plot(ls, _vcl, 'b-', alpha=0.5, label='Theory')
    ax.plot(ls, _cl, 'k-', alpha=0.5, label='Data - DICES')
    ax.plot(ls, cl1m, 'r-')
    ax.fill_between(ls, cl1m - cl1s, cl1m + cl1s, color='r', alpha=0.5, label='Delete1 - DICES')
    ax.plot(ls, cl2m, 'g-')
    ax.fill_between(ls, cl2m - cl2s, cl2m + cl2s, color='g', alpha=0.5, label='Delete2 - DICES')
    ax.legend()
    ax.set_title(f'{key}')
    ax.set_ylabel('Cls')
    ax.set_xscale('symlog', linthresh=10, linscale=0.5)
    ax.set_yscale('symlog', linthresh=1e-9, linscale=0.5)

# Hide any unused subplots
for ax in axes[len(cls1_m):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Determine the number of subplots needed
num_plots = len(cls1_m)
num_cols = math.ceil(math.sqrt(num_plots))
num_rows = math.ceil(num_plots / num_cols)

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(25, 15))

# Flatten axes array for easy iteration
axes = axes.flatten()

# Plot each Cls with error bars
for (ax, key) in zip(axes, cls1_m.keys()):
    f1, f2, b1, b2 = key
    _vcl = vcls[key]
    _cl = cls0[key]
    kk = (f1, f2, f1, f2, b1, b2, b1, b2)
    dices_c = dices_cov[kk]
    dices_err = np.sqrt(np.diag(dices_c))
    ax.plot(ls, _vcl, 'k-', alpha=0.5, label='Theory')
    ax.errorbar(ls, _cl, yerr=dices_err, color="blue", alpha=0.5, label='Data - DICES')
    ax.legend()
    ax.set_title(f'{key}')
    ax.set_ylabel('Cls')
    ax.set_yscale('log')

# Hide any unused subplots
for ax in axes[len(cls1_m):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Determine the number of subplots needed
num_plots = len(cls1_m)
num_cols = math.ceil(math.sqrt(num_plots))
num_rows = math.ceil(num_plots / num_cols)

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(25, 15))

# Flatten axes array for easy iteration
axes = axes.flatten()

# Plot each Cls with error bars
for ax, (key, value) in zip(axes, vcls.items()):
    f1, f2, b1, b2 = key
    kk = (f1, f2, f1, f2, b1, b2, b1, b2)
    c = cov1[kk]
    sc = shrunk_cov1[kk]
    c2 = cov2[kk]
    dc = dices_cov[kk]
    vc = vcov[kk]
    err = np.sqrt(np.diag(c))
    serr = np.sqrt(np.diag(sc))
    c2err = np.sqrt(np.diag(c2))
    dcerr = np.sqrt(np.diag(dc))
    verr = np.sqrt(vc)
    if f1 == "G_B":
        dcerr /= 2
    if f2 == "G_B":
        dcerr /= 2
    ax.plot(ls, verr, 'k--', lw=3,label='Gaussian Error w/ bias')
    ax.fill_between(ls, np.zeros_like(ls), err, color='green', alpha=0.3, label='delete1 cov')
    #ax.fill_between(ls, np.zeros_like(ls), serr, color='orange', alpha=0.3, label='shrunk delete1 cov')
    ax.fill_between(ls, np.zeros_like(ls), dcerr, color='blue', alpha=0.3, label='DICES cov')
    ax.fill_between(ls, np.zeros_like(ls), c2err, color='red', alpha=0.3, label='delete2 cov')
    
    ax.set_title(f'{key}')
    ax.legend()
    ax.set_ylabel('Cls')
    ax.loglog()

# Hide any unused subplots
for ax in axes[len(cls1_m):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Determine the number of subplots needed
num_plots = len(cls0)
num_cols = math.ceil(math.sqrt(num_plots))
num_rows = math.ceil(num_plots / num_cols)

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(25, 15))

# Flatten axes array for easy iteration
axes = axes.flatten()
# Plot each Cls with error bars
for ax, (key, value) in zip(axes, vcls.items()):
    f1, f2, b1, b2 = key
    kk = (f1, f2, f1, f2, b1, b2, b1, b2)
    dices_c = dices_cov[kk]
    dices_err = np.sqrt(np.diag(dices_c))
    cov1_c = cov1[kk]
    cov1_err = np.sqrt(np.diag(cov1_c))
    cov2_c = cov2[kk]
    cov2_err = np.sqrt(np.diag(cov2_c))
    vc = vcov[kk]
    verr = np.sqrt(vc)
    ax.plot(ls, np.zeros(len(ls)), color='black', alpha=0.5)
    ax.fill_between(lgrid, -0.2, 0.2, color='black', alpha=0.2)
    ax.fill_between(lgrid, -0.1, 0.1, color='black', alpha=0.2)
    ax.plot(ls, (cov1_err-verr)/verr, color='green', alpha=0.5, lw=2, label='Delete1')
    ax.plot(ls, (cov2_err-verr)/verr, color='red', alpha=0.5,lw=2, label='Delete2')
    ax.plot(ls, (dices_err-verr)/verr, color='blue', alpha=0.5, lw=2, label='DICES')
    print(key, np.mean((dices_err-verr)//dices_err))
    ax.set_title(f'{key}')
    ax.legend()
    ax.set_ylabel('Cls')
    ax.set_yscale('symlog', linthresh=0.2, linscale=1)

# Hide any unused subplots
for ax in axes[len(cls0):]:
    ax.axis('off')

plt.tight_layout()
plt.show()